# Predictor Market finance - expected return

Dans ce TP, nous allons créer un modèle de prévision d'expected return d'un portefeuille.
<br/> Le but sera d'essayer de minimiser l'écart entre nos prédictions et les valeurs observé
<br/> <b>Intérêt :</b> Anticipé les pertes afin de mieux gérer notre portefeuille

##### 1. Description du fichier

Le fichier train.csv contient les colonnes suivantes : 
<br/>Id => Id de l'asset 
<br/>Feature_i avec i de 1:25 => Des features descriptives de l'actif sans information complémentaire sur leur manière d'être calculé
<br/>Ret_MinusTwo,Ret_MinusOne => Les returns (rentabilité) connues à D-2 (Day - 2) et D-1 (Day - 1)
<br/>Ret_i avec i de 1:119 => Les returns connues pdt les deux dernières heures de la journée courante
<br/>Ret_i avec i de 12O:180 => Les returns qu'on <b>veut essayer de prédire </b>
<br/>Ret_PlusOne, Ret_PlusTwo => Les return à D+1 et D+2 qu'on <b>veut essayer de prédire</b>

##### 2.Questions

(i) Créer des fonctions permettant de:
<br/>a-Créer un jeu de train et test de manière aléatoire
<br/>b-Lancer le modèle xgboost avec un jeu de paramètre donné
<br/><b>Cette question va être corrigé</b>

(ii)Créer et sélectionner les variables à partir des données existantes pour essayer de prédire plus efficacement:
<br/>a-Ret_120
<br/>b-Ret_PlusOne
<br/>Une ébauche de réponse va être incluse => <b><u>A compléter par vos soins et me renvoyer le notebook</u> si vous choisissez ce problème</b>
<br/>Pas de problème si le résultat n'est pas sensiblement améliorer, je veux que vous m'expliquiez l'idée que vous aviez surtout!

### Création de l'environnement de développement et réponse à la première question

In [1]:
#Inclusion de toutes les librairies qui seront nécessaire 
# xgboost => librairie optimisé utilisant le model de descente du gradient optimisé: http://xgboost.readthedocs.io/en/latest/model.html 
import xgboost as xgb
# train_test_split => Fonctionnalité très utile pour créer ses datasets de test et de train
from sklearn.model_selection import train_test_split
# pandas => Librairie pour manipulé toute sorte de tableau
import pandas as pd
# numpy => librairie pour les agrégats et tableau avec des valeurs numériques 
# (modélisation différente entre les arrays numpy et les dataframes pandas)
import numpy as np

C:\Users\pfisz010614\AppData\Roaming\Python\Python35\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Le path vers le fichier que vous avez en local
filepath = 'D:/Formation/Dauphine/TP2/Market_Finance/train.csv' 

In [3]:
# Fonctionnalité pour les csv: pd.read_csv pr excel c'est pd.read_excel 
# Il y a de multiples paramètrages possible 
df_train = pd.read_csv(filepath, sep=',', decimal='.')

In [4]:
#fonction utile pour pouvoir rapidement faire la selection de columns qu'on veut garder pr réduire le dataframe
def print_columns_dataframe(df):
    for value in df.columns:     
        print("'" + value + "',")

In [5]:
# Calcul de erreur au carre
def erreur_carre(preds, y, weight):
    ylist = y.tolist()
    erreur = []
    sumresult_ = 0
    for i in range(0, len(preds)):
        er_tmp = weight[i] * (preds[i] - ylist[i])**2
        erreur.append(er_tmp)
        sumresult_ = sumresult_ + er_tmp
    return erreur, sumresult_

In [6]:
#Calcul de l'erreur absolue
def erreur_absolu(preds, y, weight):
    ylist = y.tolist()
    erreur = []
    sumresult_ = 0
    for i in range(0, len(preds)):
        er_tmp = weight[i] *  abs(preds[i] - ylist[i])
        erreur.append(er_tmp)
        sumresult_ = sumresult_ + er_tmp
    return erreur, sumresult_

In [7]:
def calcul_model(dataset, test_size, y, param, num_round, rnd):
    X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size=test_size, random_state=rnd)
    dtrain = xgb.DMatrix(X_train, y_train)
    dtest = xgb.DMatrix(X_test, y_test)
    bst = xgb.train(param, dtrain, num_round)
    # make prediction
    preds = bst.predict(dtest)
    return y_test, preds, bst

In [8]:
def calcul_model_erreur(prediction, y_output, weight, error_type):
    if error_type == 'absolue':
        erreur, result = erreur_absolu(prediction, y_output, equal_weight)
    elif error_type == 'carre':
        erreur, result = erreur_carre(prediction, y_output, equal_weight)
    else:
        print('erreur pour le paramètre error_type: ' + error_type) 
        return None
    df_erreur = pd.DataFrame(np.transpose(erreur), columns=['erreur'])
    df_output = pd.concat([pd.DataFrame(np.transpose(y_output.tolist()), columns=['y_output']), pd.DataFrame(np.transpose(pred), columns=['preds']),df_erreur], axis=1)
    return df_output

In [9]:
print_columns_dataframe(df_train)

'Id',
'Feature_1',
'Feature_2',
'Feature_3',
'Feature_4',
'Feature_5',
'Feature_6',
'Feature_7',
'Feature_8',
'Feature_9',
'Feature_10',
'Feature_11',
'Feature_12',
'Feature_13',
'Feature_14',
'Feature_15',
'Feature_16',
'Feature_17',
'Feature_18',
'Feature_19',
'Feature_20',
'Feature_21',
'Feature_22',
'Feature_23',
'Feature_24',
'Feature_25',
'Ret_MinusTwo',
'Ret_MinusOne',
'Ret_2',
'Ret_3',
'Ret_4',
'Ret_5',
'Ret_6',
'Ret_7',
'Ret_8',
'Ret_9',
'Ret_10',
'Ret_11',
'Ret_12',
'Ret_13',
'Ret_14',
'Ret_15',
'Ret_16',
'Ret_17',
'Ret_18',
'Ret_19',
'Ret_20',
'Ret_21',
'Ret_22',
'Ret_23',
'Ret_24',
'Ret_25',
'Ret_26',
'Ret_27',
'Ret_28',
'Ret_29',
'Ret_30',
'Ret_31',
'Ret_32',
'Ret_33',
'Ret_34',
'Ret_35',
'Ret_36',
'Ret_37',
'Ret_38',
'Ret_39',
'Ret_40',
'Ret_41',
'Ret_42',
'Ret_43',
'Ret_44',
'Ret_45',
'Ret_46',
'Ret_47',
'Ret_48',
'Ret_49',
'Ret_50',
'Ret_51',
'Ret_52',
'Ret_53',
'Ret_54',
'Ret_55',
'Ret_56',
'Ret_57',
'Ret_58',
'Ret_59',
'Ret_60',
'Ret_61',
'Ret_62',
'Ret_63',
'Ret_64',

In [9]:
#Exclusion de l'Id et de toutes les colonnes à prédire
dataset = df_train.drop(columns=['Ret_120','Ret_121','Ret_122','Ret_123','Ret_124','Ret_125','Ret_126','Ret_127','Ret_128','Ret_129',
'Ret_130','Ret_131','Ret_132','Ret_133','Ret_134','Ret_135','Ret_136','Ret_137','Ret_138','Ret_139','Ret_140','Ret_141','Ret_142','Ret_143','Ret_144','Ret_145','Ret_146','Ret_147','Ret_148','Ret_149','Ret_150','Ret_151','Ret_152','Ret_153','Ret_154','Ret_155','Ret_156','Ret_157','Ret_158','Ret_159','Ret_160','Ret_161','Ret_162','Ret_163','Ret_164','Ret_165','Ret_166','Ret_167','Ret_168',
'Ret_169','Ret_170','Ret_171','Ret_172','Ret_173','Ret_174','Ret_175','Ret_176','Ret_177','Ret_178','Ret_179','Ret_180','Ret_PlusOne', 'Ret_PlusTwo','Weight_Intraday','Weight_Daily']) #'Id',
dataset_no_id = dataset.drop(columns='Id') 
dataset_no_id = dataset_no_id.fillna(0)

In [10]:
#Toutes les rentabilités que l'on pourrait vouloir essayer de résoudre
y_ret_120 = df_train['Ret_120']
y_ret_180 = df_train['Ret_180']
y_ret_d_1 = df_train['Ret_PlusOne']
y_ret_d_2 =df_train['Ret_PlusTwo']

In [11]:
# Les poids qui seront nécessaire pour le calcul de la fonction d'erreur
weight_intra = df_train['Weight_Intraday'].tolist() # à utiliser pour prédire Ret_120 & Ret_180
weight_daily = df_train['Weight_Daily'].tolist()    # à utiliser pour prédire Ret_PlusOne & Ret_PlusTwo
equal_weight = [1] * len(weight_daily) # pour faire nos test avant de chercher à optimiser

In [12]:
# Un jeu de paramètre à pour le modèle xgboos
# max_depth : Profondeur max des arbres
# eta : coefficient de la courbe d'apprentissage (learning_curve)
# silent : verbose
# tous les paramètres sont détaillés ici http://xgboost.readthedocs.io/en/latest/parameter.html
param = {'max_depth':6, 'eta':0.225, 'silent': 0, 'objective':'reg:linear', 'eval_metric': 'mae','lambda':1.5} #,'gamma':1.5
y_output, pred, bst = calcul_model(dataset_no_id, 0.33, y_ret_120, param, 400, 42)

In [13]:
#calcul de l'erreur avec la fonction d'erreur absolue
df_output_absolue = calcul_model_erreur(pred, y_output, equal_weight, 'absolue')
df_output_absolue.erreur.sum()

7.922132918179315

In [14]:
# calcul de l'erreur avec la fonction d'erreur au carree
param_carre = {'max_depth':6, 'eta':0.225, 'silent': 0, 'objective':'reg:linear', 'eval_metric': 'rmse','lambda':1.5} #,'gamma':1.5
df_output_carre = calcul_model_erreur(pred, y_output, equal_weight, 'carre')
df_output_carre.erreur.sum()

0.023223128276449508

In [15]:
df_output_test = calcul_model_erreur(pred, y_output, equal_weight, 'blabla')
type(df_output_test)

erreur pour le paramètre error_type: blabla


NoneType

In [17]:
param_180 = {'max_depth':6, 'eta':0.225, 'silent': 0, 'objective':'reg:linear', 'eval_metric': 'mae','lambda':1.5} #,'gamma':1.5
y_output_180, pred_180, bst_180 = calcul_model(dataset_no_id, 0.33, y_ret_180, param_180, 400, 42)
df_output_absolue_180 = calcul_model_erreur(pred_180, y_output_180, equal_weight, 'absolue')
df_output_absolue_180.erreur.sum()

9.414808717998726

In [18]:
param_d_1 = {'max_depth':6, 'eta':0.225, 'silent': 0, 'objective':'reg:linear', 'eval_metric': 'rmaemse','lambda':1.5} #,'gamma':1.5
y_output_d_1, pred_d_1, bst_d_1 = calcul_model(dataset_no_id, 0.33, y_ret_d_1, param_d_1, 400, 42)
df_output_absolue = calcul_model_erreur(pred_d_1, y_output_d_1, equal_weight, 'absolue')
df_output_absolue.erreur.sum()

201.76110391805906

### Question 2

Nous allons utilisé comme valeur de référence la prédiction que nous avons obtenu pour la question 1 pour y_180 et y_d_1.
<br/>Il y a deux axes pour améliorer la prédiction : 
<br/><ul>1.optimiser les paramètres de xgboost
<br/>2.Transformer les variables afin d'obtenir plus d'information</ul>
Une idée de variable va vous être proposé et vous aurez à compléter la question/créer de nouvelles variables/modifier les paramètres de xgboost

Ci-dessous est des exemples de bout de code pour créer des nouvelles variables ou des sous selection de variables

In [ ]:
#Comptage du nombre de return positive / négative
unpivot_Ret = pd.melt(df_train,value_name='Return_time',value_vars= ['Ret_2', 'Ret_3', 'Ret_4', 'Ret_5', 'Ret_6', 'Ret_7', 'Ret_8', 'Ret_9', 'Ret_10', 'Ret_11', 'Ret_12', 'Ret_13', 'Ret_14', 'Ret_15', 'Ret_16', 'Ret_17', 'Ret_18', 'Ret_19', 'Ret_20', 'Ret_21', 'Ret_22', 'Ret_23', 'Ret_24', 'Ret_25', 'Ret_26', 'Ret_27', 'Ret_28', 'Ret_29', 'Ret_30', 'Ret_31', 'Ret_32', 'Ret_33', 'Ret_34', 'Ret_35', 'Ret_36', 'Ret_37', 'Ret_38', 'Ret_39', 'Ret_40', 'Ret_41', 'Ret_42', 'Ret_43', 'Ret_44', 'Ret_45', 'Ret_46', 'Ret_47', 'Ret_48', 'Ret_49', 'Ret_50', 'Ret_51', 'Ret_52', 'Ret_53', 'Ret_54', 'Ret_55', 'Ret_56', 'Ret_57', 'Ret_58', 'Ret_59', 'Ret_60', 'Ret_61', 'Ret_62', 'Ret_63', 'Ret_64', 'Ret_65', 'Ret_66', 'Ret_67', 'Ret_68', 'Ret_69', 'Ret_70', 'Ret_71', 'Ret_72', 'Ret_73', 'Ret_74', 'Ret_75', 'Ret_76', 'Ret_77', 'Ret_78', 'Ret_79', 'Ret_80', 'Ret_81', 'Ret_82', 'Ret_83', 'Ret_84', 'Ret_85', 'Ret_86', 'Ret_87', 'Ret_88', 'Ret_89', 'Ret_90', 'Ret_91', 'Ret_92', 'Ret_93', 'Ret_94', 'Ret_95', 'Ret_96', 'Ret_97', 'Ret_98', 'Ret_99', 'Ret_100', 'Ret_101', 'Ret_102', 'Ret_103', 'Ret_104', 'Ret_105', 'Ret_106', 'Ret_107', 'Ret_108', 'Ret_109', 'Ret_110', 'Ret_111', 'Ret_112', 'Ret_113', 'Ret_114', 'Ret_115', 'Ret_116', 'Ret_117', 'Ret_118', 'Ret_119'], id_vars='Id')
unpivot_Ret['Sens_return'] = np.where(unpivot_Ret.Return_time < 0, 'Number of Negative' , 'Number of positive')

In [15]:
#Calcul d'indicateur simple sur les return de 2 à 119
ret_number = pd.pivot_table(unpivot_Ret, values ='Return_time', columns='Sens_return', index='Id', aggfunc=len)
ret_number = ret_number.reset_index()
ret_stat = pd.pivot_table(unpivot_Ret, values ='Return_time', index='Id', aggfunc=[np.mean, np.min, np.max, np.std, np.median])
ret_stat.columns = ["|".join((j,k)) for j,k in ret_stat.columns]
ret_stat = ret_stat.reset_index()

In [ ]:
#Merge des nouvelles variables calculées
dataset = dataset.merge(ret_stat, on='Id', how='inner')
dataset = dataset.merge(ret_number, on='Id', how='inner')

In [28]:
y_day_one = df_train['Ret_PlusOne']

In [39]:
#Selection d'un sous dataset
dataset_2 = dataset[['Feature_1','Feature_2', 'Feature_3','Feature_4', 'Feature_5','Feature_6',
                   'Feature_7','Feature_8', 'Feature_9','Feature_10','Feature_11','Feature_12',
                   'Feature_13','Feature_14','Feature_15','Feature_16','Feature_17','Feature_18',
                   'Feature_19','Feature_20', 'Feature_21','Feature_22','Feature_23','Feature_24',
                   'Feature_25','Ret_MinusTwo','Ret_MinusOne','Ret_2','Ret_60', 'Ret_109', 'Ret_110', 'Ret_111', 
                    'Ret_112', 'Ret_113', 'Ret_114', 'Ret_115', 'Ret_116', 'Ret_117', 'Ret_118','Ret_119',
                   'mean|Return_time', 'Number of Negative','Number of positive', 'std|Return_time','median|Return_time']]
# 'amin|Return_time', 'amax|Return_time',

In [54]:
#Division par exemple du problème en deux partie (débile car je dis diviser selon day-1... mais il y a surement mieux à faire :) )
dataset_2_neg = dataset_2[dataset_2.Ret_MinusOne<0]
y_neg = df_train[df_train.Ret_MinusOne<0]['Ret_PlusOne']
dataset_2_pos = dataset_2[dataset_2.Ret_MinusOne>0]
y_pos = df_train[df_train.Ret_MinusOne>0]['Ret_PlusOne']

In [40]:
# Modèle avec la valeur de référence qu'on va essayer de battre
param_d_1 = {'max_depth':5, 'eta':0.1, 'silent': 0, 'objective':'reg:linear', 'eval_metric': 'mae','lambda':1.5} #,'gamma':1.5
y_output_d_1, pred_d_1, bst_d_1 = calcul_model(dataset_no_id, 0.33, y_day_one, param_d_1, 150, 42)
df_output_absolue_d_1 = calcul_model_erreur(pred_d_1, y_output_d_1, equal_weight, 'absolue')
df_output_absolue_d_1.erreur.sum()

200.9744011113247

In [30]:
#calcul du modèle avec le sous dataset
param_d_1 = {'max_depth':5, 'eta':0.1, 'silent': 0, 'objective':'reg:linear', 'eval_metric': 'mae','lambda':1.5} #,'gamma':1.5
y_output_d_1, pred_d_1, bst_d_1 = calcul_model(dataset_2, 0.33, y_day_one, param_d_1, 150, 42)
df_output_absolue_d_1 = calcul_model_erreur(pred_d_1, y_output_d_1, equal_weight, 'absolue')
df_output_absolue_d_1.erreur.sum()

199.38490953963412

In [55]:
#calcul du modèle divisé en 2 (1/2)
param_d_1 = {'max_depth':5, 'eta':0.1, 'silent': 0, 'objective':'reg:linear', 'eval_metric': 'mae','lambda':1.5} #,'gamma':1.5
y_output_d_1_neg, pred_d_1_neg, bst_d_1_neg = calcul_model(dataset_2_neg, 0.33, y_neg, param_d_1, 150, 42)
df_output_absolue_d_1_neg = calcul_model_erreur(pred_d_1_neg, y_output_d_1_neg, equal_weight, 'absolue')
df_output_absolue_d_1_neg.erreur.sum()

100.44475495877205

In [56]:
#calcul du modèle divisé en 2 (2/2)
param_d_1 = {'max_depth':5, 'eta':0.1, 'silent': 0, 'objective':'reg:linear', 'eval_metric': 'mae','lambda':1.5} #,'gamma':1.5
y_output_d_1_pos, pred_d_1_pos, bst_d_1_pos = calcul_model(dataset_2_pos, 0.33, y_pos, param_d_1, 150, 42)
df_output_absolue_d_1_pos = calcul_model_erreur(pred_d_1_pos, y_output_d_1_pos, equal_weight, 'absolue')
df_output_absolue_d_1_pos.erreur.sum()

99.49453736952788

Maintenant c'est à vous de jouer : Reprendre toutes les parties du code que vous voulez pour essayer de prédire mieux que le modèle de référence (vous pouvez utiliser les poids que vous préférez)